## Towards a final dataset

- Create final dataset from existing scripts

Environment

In [47]:
import pandas as pd

In [48]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [49]:
import re

In [50]:
import dateutil

Original data

In [51]:
data = pd.read_csv('20190313_TW_Migrant-Migration.csv')
data[0:1]

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,user_location,status_url,entities_str
0,1.102720e+18,darleneclarke5,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:47 +0000 2019,04/03/2019 23:59:47,NaN,en,NaN,NaN,564257648.0,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://abs.twimg.com/sticky/default_profile_im...,197,579.0,NaN,http://twitter.com/darleneclarke5/statuses/110...,"{""hashtags"":[],""symbols"":[],""user_mentions"":[{..."


In [52]:
#New df
df=data

Current state of the dataframe

In [72]:
df[0:1]

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetMethod,tweetSentiment,tweetText,tweetTimeDate,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new
0,197,579.0,564257648.0,NaN,en,NaN,http://abs.twimg.com/sticky/default_profile_im...,darleneclarke5,NaN,0,...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",-0.9403,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:47 +0000 2019,http://twitter.com/darleneclarke5/statuses/110...,English,Default,Web_Standard,Mon Mar 04 2019,2019-03-04


Unique ID variable for each tweet

In [54]:
df.reset_index(inplace=True)
df.rename(columns={'index':'tweetID'}, inplace=True)

Sentiment Score

In [55]:
map_ID_text = dict((tuple(x) for x in df[['tweetID', 'text']].values))

In [56]:
analyzer = SentimentIntensityAnalyzer()

map_ID_valence_dict = {}

for ID, text in map_ID_text.items():
    map_ID_valence_dict[ID] = analyzer.polarity_scores(text)

In [57]:
map_ID_senti_score = {} # Mapping IDs to a single score 'combined'

for ID, val in map_ID_valence_dict.items():
    map_ID_senti_score[ID] = val['compound']

In [58]:
df['SentiScore_VADER'] = df['tweetID'].map(map_ID_senti_score)

Drop unnecessary columns

In [60]:
#Drop the "text" and "entities" columns that Sunyam is working on. 
#Also dropping time because it is redundant with "created_at"
df = df.drop(['id_str','time','entities_str', 'geo_coordinates'], axis=1)

In [61]:
#Rename columns
df = df.rename(columns={"id_str":"tweetID",
                        "from_user":"authorScreen",
                        "created_at":"tweetTimeDate",
                        "geo_coordinates":"tweetGeo",
                        "user_lang":"authorLang",
                        "in_reply_to_user_id_str":"authorID_original",
                        "in_reply_to_screen_name":"authorScreen_original",
                        "from_user_id_str":"authorID",
                        "in_reply_to_status_id_str":"tweetID_original",
                        "source":"tweetMethod", 
                        "profile_image_url":"authorPicture",
                        "user_followers_count":"authorFollower",
                        "user_friends_count":"authorFriend",
                        "user_location":"authorLocation",
                        "status_url":"tweetURL",
                        "text":"tweetText",
                        "SentiScore_VADER": "tweetSentiment"})

In [62]:
#Group author and tweet variables together in the dataset.
df = df.sort_index(axis=1)

Recode variables

In [64]:
df["authorLang_new"] = df["authorLang"].astype('category')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'en.*', value='English')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'^((?!English)\S*)', value='Other')
df['authorLang_new'].value_counts()

English    109347
Other       14177
Name: authorLang_new, dtype: int64

In [65]:
df["authorPicture_new"] = df["authorPicture"].astype('category')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*default_profile_images\S*', value='Default')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*/profile_images\S*', value='Personalized')
df['authorPicture_new'].value_counts()

Personalized    117131
Default           6393
Name: authorPicture_new, dtype: int64

In [66]:
df["tweetMethod_new"] = df["tweetMethod"].astype('category')

In [67]:
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPhone\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for Android\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web Client\S*', value='Web_Standard')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web App\S*', value='Web_Lite')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPad\S*', value='Web_Lite')
df['tweetMethod_new'].value_counts()[0:10]

Smartphone                                                                             74524
Web_Standard                                                                           24144
Web_Lite                                                                               16764
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                   1300
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>     1079
<a href="https://ifttt.com" rel="nofollow">IFTTT</a>                                     500
<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>                                 474
<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>                    457
<a href="https://buffer.com" rel="nofollow">Buffer</a>                                   360
<a href="http://publicize.wp.com/" rel="nofollow">WordPress.com</a>                      266
Name: tweetMethod_new, dtype: int64

In [68]:
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*href.*', value='Other')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^Web_Standard.*(T1).*', value='Other')
df['tweetMethod_new'].value_counts()

Smartphone      74524
Web_Standard    24144
Web_Lite        16764
Other            8092
Name: tweetMethod_new, dtype: int64

In [69]:
df['tweetDate'] = df['tweetTimeDate'].replace(regex=r'\s\d{2}:.*:\d{2}\s.{5}', value='')
df['tweetDate_new'] = df['tweetDate'].apply(dateutil.parser.parse, dayfirst=True)

Smartphone      74524
Web_Standard    24144
Web_Lite        16764
Other            8092
Name: tweetMethod_new, dtype: int64

In [70]:
df['tweetDate_new'].value_counts()[0:9]

2019-02-28    2913
2019-01-26    2910
2019-02-22    2909
2019-02-10    2906
2019-02-04    2905
2019-02-08    2902
2019-02-11    2901
2019-01-28    2900
2019-03-01    2897
Name: tweetDate_new, dtype: int64

In [71]:
df.to_csv('20190402_AM_Dataset.csv',index=False)